This notebook completes the PanTaGruEl model as found at [https://github.com/laurentpagnier/PanTaGruEl.jl](https://github.com/laurentpagnier/PanTaGruEl.jl)

# Import model

In [1]:
import os
import json

In [2]:
with open('../pantagruel.json') as f:
    model = json.load(f)

# Fix missing bus locations

In [3]:
bus_location_fix = {
    "7321": "FR",
    "7578": "FR",
    "7015": "FR",
    "7586": "FR",
    "7788": "FR",
    "1482": "IT",
    "7822": "FR",
    "7380": "FR",
    "7336": "FR",
    "7333": "FR",
    "7327": "FR",
    "7175": "PT",
    "6988": "DK",
    "7334": "FR",
    "6873": "DK",
    "7821": "FR",
    "7575": "FR",
    "7750": "FR",
    "7337": "FR",
    "7715": "FR",
    "7339": "FR",
    "7741": "FR",
    "6949": "FR",
    "6943": "FR",
    "7322": "FR",
    "7326": "FR"
}

In [4]:
for (id, country) in bus_location_fix.items():
    model["bus"][id]["country"] = country

# Add ENTSO-E generator names

Add temporary unique generator name for all gens of the model, as well as the generators country:

In [5]:
for gen_id, gen in model['gen'].items():
    bus_id = str(gen['gen_bus'])
    gen['name'] = model['bus'][bus_id]['name'] + ' ' + gen_id
    gen['country'] = model['bus'][bus_id]['country']

In [6]:
all_gen_names = {gen['name'] for gen in model['gen'].values()}

Import list of generators from ENTSO-E at a given reference time (January 2016)

In [7]:
import pandas as pd

In [8]:
entsoe_data_dir = os.path.expanduser('~/data/entso-e/raw')

In [9]:
entsoe_data = pd.DataFrame(pd.read_csv('%s/2016_01_ActualGenerationOutputPerGenerationUnit_16.1.A.csv' % entsoe_data_dir, sep='\t') \
    .groupby(['PowerSystemResourceName', 'MapCode', 'ProductionType', 'InstalledGenCapacity'])['ActualGenerationOutput'].mean()) \
    .reset_index()
entsoe_data

,PowerSystemResourceName,MapCode,ProductionType,InstalledGenCapacity,ActualGenerationOutput
0,ABOÑO 1,ES,Fossil Hard coal,341.7,155.520027
1,ABOÑO 2,ES,Fossil Hard coal,535.8,411.376075
2,ABTH7,GB,Fossil Hard coal,535.0,501.421135
3,ABTH8,GB,Fossil Hard coal,535.0,483.604071
4,ABTH9,GB,Fossil Hard coal,535.0,395.362667
...,...,...,...,...,...
1679,Zydowo H3,PL,Hydro Pumped Storage,54.0,2.414234
1680,iPower Solutions ltd,NIE,Fossil Oil,74.0,0.034341
1681,Åbyverket Örebro,SE,Biomass,106.0,29.132615
1682,Öresundsverket CHP G1,SE,Fossil Gas,294.0,210.140970


Determine country from MapCode column:

In [10]:
entsoe_country_map = {mapcode: mapcode for mapcode in entsoe_data['MapCode']}
for mapcode in entsoe_country_map.keys():
    if mapcode[0:2] == 'DE':
        entsoe_country_map[mapcode] = 'DE'
entsoe_data.insert(2, 'Country', list(map(lambda mapcode: entsoe_country_map[mapcode], entsoe_data['MapCode'])))

Filter generators that are in one of the model's country (ENTSO-E data also containes UK, Ireland, Sweden...)

In [11]:
model_countries = {gen['country'] for gen in model['gen'].values()}
model_countries

{'AL',
 'AT',
 'BA',
 'BE',
 'BG',
 'CH',
 'CZ',
 'DE',
 'DK',
 'ES',
 'FR',
 'GR',
 'HR',
 'HU',
 'IT',
 'LU',
 'ME',
 'MK',
 'NL',
 'PL',
 'PT',
 'RO',
 'RS',
 'SI',
 'SK',
 'XX'}

In [12]:
entsoe_data = entsoe_data[entsoe_data['Country'].isin(model_countries)]

### Switzerland

Manually examine ENTSO-E generators list:

In [13]:
CH_name_map = {
    'Beznau 943': ['Beznau 1', 'Beznau 2'],
    'Mühleberg 917': ['KKM Produktion'],
    'Leibstadt 908': ['Leibstadt'],
    'Gösgen 921': ['Kernkraftwerk Gösgen'],
    'Biasca 938': ['Centrale di Biasca'],
    'Veytaux 913': ['Usine de Veytaux'],
    'Bâtiaz 912': ['Usine de la Bâtiaz'],
    'Chamoson 914': ['Usine de Bieudron', 'Usine de Nendaz'],
    'Fionnay GD 911': ['Usine de Fionnay'],
    'Verbano 922': ['Centrale di Verbano'],
    'Robiei 936': ['Centrale di Robiei'],
    'Bitsch 916': ['Kraftwerk Bitsch'],
    'Bavona 935': ['Centrale di Bavona'],
    'Cavergno 923': ['Centrale di Cavergno'],
    'Mapragg 939': ['Mapragg - Gigerwald G1'],
    'Grimsel 909': ['KWO Produktion'],
    'Innertkirchen 918': ['KWO Produktion'],
    'Riddes 173': ['Stufe FMM Produktion Total'],
    'Fionnay FMM 910': ['Stufe FMM Produktion Total'],
    'Limmern 958': ['Limmern - Muttsee G1'],
    'Tierfehd 926': ['Tierfehd - Limmern GPSW'],
    'Olivone 925': ['AET Leventina']
}

In [14]:
CH_name_values = {name for list_of_names in CH_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [15]:
assert set(CH_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [16]:
assert CH_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [17]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'CH' and gen['name'] not in CH_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
11,Bärenburg 930,hydro_pure_storage,220.0
0,Castasegna 928,hydro_pure_storage,100.0
3,Ferrera 929,hydro_mixed,180.0
9,Foretaille 766,hydro_ror,98.0
5,Göschenen 919,hydro_pure_storage,160.0
6,Handeck 937,hydro_mixed,113.0
2,Laufenburg 920,hydro_ror,106.0
12,Löbbia 933,hydro_pure_storage,95.0
1,Pradella 934,hydro_pure_storage,288.0
4,Rothenbrunnen 932,hydro_pure_storage,127.0


ENTSO-E generators not in the list:

In [18]:
entsoe_data[(entsoe_data['Country'] == 'CH') & (~entsoe_data['PowerSystemResourceName'].isin(CH_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
776,KW Rheinfelden CH,Hydro Run-of-river and poundage,100.0
1580,Usine de Vallorcine,Hydro Water Reservoir,230.0


### France

In [19]:
FR_name_map = {
    'Belleville 757': ['BELLEVILLE 1', 'BELLEVILLE 2'],
    'Boctois 715': ['NOGENT 1', 'NOGENT 2'],
    'Cattenom 732': ['CATTENOM 1', 'CATTENOM 2', 'CATTENOM 3', 'CATTENOM 4'],
    'Chinon B 751': ['CHINON 1', 'CHINON 2', 'CHINON 3', 'CHINON 4'],
    'Chooz 518': ['CHOOZ 1', 'CHOOZ 2'],
    'Civaux 602': ['CIVAUX 1', 'CIVAUX 2'],
    'Coulange 657': ['CRUAS 1', 'CRUAS 2', 'CRUAS 3', 'CRUAS 4'],
    'Creys 690': ['BUGEY 2', 'BUGEY 3', 'BUGEY 4', 'BUGEY 5'],
    'Dampierre 708': ['DAMPIERRE 1', 'DAMPIERRE 2', 'DAMPIERRE 3', 'DAMPIERRE 4'],
    'Fessenheim 318': ['FESSENHEIM 1', 'FESSENHEIM 2'],
    'Flamanville 982': ['FLAMANVILLE 1', 'FLAMANVILLE 2'],
    'Golfech 552': ['GOLFECH 1', 'GOLFECH 2'],
    'Gravelines 723': ['GRAVELINES 1', 'GRAVELINES 2', 'GRAVELINES 3', 'GRAVELINES 4', 'GRAVELINES 5', 'GRAVELINES 6'],
    'Le Blayais 583': ['BLAYAIS 1', 'BLAYAIS 2', 'BLAYAIS 3', 'BLAYAIS 4'],
    'Paluel 718': ['PALUEL 1', 'PALUEL 2', 'PALUEL 3', 'PALUEL 4'],
    'Penly Poste 717': ['PENLY 1', 'PENLY 2'],
    'P.Cordier 670': ['ST ALBAN 1', 'ST ALBAN 2'],
    'St. Laurent 554': ['ST LAURENT 1', 'ST LAURENT 2'],
    'Tricastin 658': ['TRICASTIN 1', 'TRICASTIN 2', 'TRICASTIN 3', 'TRICASTIN 4'],
    'Cordemais Poste 986': ['CORDEMAIS 2', 'CORDEMAIS 3', 'CORDEMAIS 4', 'CORDEMAIS 5'],
    'Porcheville 711': ['PORCHEVILLE 1', 'PORCHEVILLE 2', 'PORCHEVILLE 3', 'PORCHEVILLE 4'],
    'Aramon 556': ['ARAMON 1', 'ARAMON 2'],
    'Villarodin 695': ['VILLARODIN 1', 'VILLARODIN 2'],
    'Le Pouget 553': ['POUGET 4'],
    'Brommat 758': ['BROMMAT 7'],
    'Le Cheylas 685': ['CHEYLAS 1', 'CHEYLAS 2'],
    'S. Bissorte 765': ['SUPER BISSORTE 1', 'SUPER BISSORTE 2', 'SUPER BISSORTE 3', 'SUPER BISSORTE 4', 'SUPER BISSORTE 5'],
    'Montezic poste et centrale 759': ['MONTEZIC 1', 'MONTEZIC 2', 'MONTEZIC 3', 'MONTEZIC 4'],
    'Revin 726': ['REVIN 1', 'REVIN 2', 'REVIN 3', 'REVIN 4'],
    'Guersac 987': ['SPEM CCG'],
    'Quartes 725': ['Pont-sur-Sambre'],
    'Braek 555': ['DK6-TG1', 'DK6-TG2', 'DK6-TV1', 'DK6-TV2'],
    'Ratier 754': ['AMFARD14', 'AMFARD15'],
    'Darse 610': ['COMBIGOLFE CCG', 'CYCOFOS PL1', 'CYCOFOS PL2'],
    'Ponteau 955': ['MARTIGUES PONTEAU 5', 'MARTIGUES PONTEAU 6'],
    'Custines 749': ['Croix-de-Metz'],
    'St. Avold 731': ['EMILE HUCHET 6', 'EMILE HUCHET 7', 'EMILE HUCHET 8'],
    'Bezaumont 729': ['BLENOD 5'],
    'Le Havre 603': ['HAVRE 4'],
    'La Palun 672': ['PROVENCE 5'],
    'Arrighi 709': ['ARRIGHI 1', 'ARRIGHI 2'],
    'Brennilis 985': ['BRENNILIS 1', 'BRENNILIS 2', 'BRENNILIS 3'],
    'Loscoat 983': ['DIRINON 1', 'DIRINON 2'],
    'Le Chesnoy 714': ['MONTEREAU 5', 'MONTEREAU 6'],
    'Villevaude 716': ['VAIRES 1', 'VAIRES 2', 'VAIRES 3'],
    'Sisteron 677': ['SISTERON 1', 'SISTERON 2'],
    'Le Chastang 612': ['CHASTANG 2', 'CHASTANG 3']
}

In [20]:
FR_name_values = {name for list_of_names in FR_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [21]:
assert set(FR_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [22]:
assert FR_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [23]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'FR' and gen['name'] not in FR_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
73,Airvault 597,Biomass,2.1260
72,Amiens 720,Biomass,2.8300
122,Amiens 721,Biomass,1.9030
158,Arnage 601,Biomass,14.3750
153,Aston 557,hydro_ror,104.0000
...,...,...,...
59,Vignol 668,Hydro,6.2600
95,Vincey 727,Hydro,1.5009
56,Vogelgrun 737,hydro_ror,141.0000
134,Volvon 762,Hydro,2.7200


ENTSO-E generators not in the list:

In [24]:
entsoe_data[(entsoe_data['Country'] == 'FR') & (~entsoe_data['PowerSystemResourceName'].isin(FR_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
15,AIGLE 6,Hydro Water Reservoir,146.0
118,BORT 1,Hydro Water Reservoir,118.0
119,BORT 2,Hydro Water Reservoir,114.0
245,COMBE D'AVRIEUX 1,Hydro Water Reservoir,123.0
548,FR-GA-MORANT1,Fossil Gas,401.0
569,GENNEVILLIERS 1,Fossil Gas,203.0
585,GRAND MAISON 1,Hydro Pumped Storage,152.0
586,GRAND MAISON 10,Hydro Pumped Storage,157.0
587,GRAND MAISON 11,Hydro Pumped Storage,157.0
588,GRAND MAISON 12,Hydro Pumped Storage,157.0


### Spain

In [25]:
ES_name_map = {
    'Asco 79': ['ASCO 1', 'ASCO 2'],
    'C.N. Almaraz 160': ['ALMARAZ 1', 'ALMARAZ 2'],
    'Sagunto 87': ['COFRENTES'],
    'La Robla 1005': ['ROBLA 1', 'ROBLA 2'],
    'Soto de Ribera 1013': ['SOTO RIB 2', 'SOTO RIB 3'],
    'P.G. Rodriguez 998': ['P.G.RODR 1', 'P.G.RODR 2', 'P.G.RODR 3', 'P.G.RODR 4'],
    'Teruel 77': ['TERUEL 1', 'TERUEL 2', 'TERUEL 3'],
    'Lada 1009': ['LADA IV'],
    'Compostilla I 1004': ['ANLLARES 1', 'COMPOSTI 4', 'COMPOSTI 5'],
    'Belesar 997': ['BELESAR 1', 'BELESAR 2', 'BELESAR 3'],
    'G. y Galan 152': ['GGALAN GEN'],
}

In [26]:
ES_name_values = {name for list_of_names in ES_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [27]:
assert set(ES_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [28]:
assert ES_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [29]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'ES' and gen['name'] not in ES_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
56,Aguayo 1010,Hydro,360.60
41,Aldeadávila II 166,hydro_ror,820.00
60,Alvarado 158,other_nl,150.00
66,Andújar 143,Waste,15.00
71,Aravaca 161,Hydro,61.95
...,...,...,...
48,Vandellos 80,Gas,39.58
82,Vilecha 1006,Hydro,34.68
17,Villalcampo 156,hydro_ror,881.00
9,Villamayor 154,other_nl,120.00


ENTSO-E generators not in the list:

In [30]:
entsoe_data[(entsoe_data['Country'] == 'ES') & (~entsoe_data['PowerSystemResourceName'].isin(ES_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
0,ABOÑO 1,Fossil Hard coal,341.7
1,ABOÑO 2,Fossil Hard coal,535.8
5,ACECA 3,Fossil Gas,386.0
6,ACECA4,Fossil Gas,372.6
16,ALDEA I G1,Hydro Water Reservoir,132.9
...,...,...,...
1601,VLLRINO 4G,Hydro Water Reservoir,138.7
1602,VLLRINO 5B,Hydro Pumped Storage,138.0
1603,VLLRINO 5G,Hydro Water Reservoir,148.9
1604,VLLRINO 6B,Hydro Pumped Storage,138.0


### Germany

In [31]:
DE_name_map = {
    'Brokdorf 210': ['Brokdorf'],
    'Gundremmingen 256': ['Gundremmingen B'],
    'Gundremmingen 257': ['Gundremmingen C'],
    'Isar 265': ['Isar 2'],
    'Philippsburg 232': ['KKW Philippsburg 2'],
    'Grohnde 209': ['Grohnde'],
    'Emsland 249': ['Emsland A'],
    'GKN 2 950': ['KKW Neckarwestheim 2'],
    'Boxberg 314': ['KW Boxberg Block N', 'KW Boxberg Block P', 'KW Boxberg Block Q', 'KW Boxberg Block R'],
    'Jänschwalde 312': ['KW Jänschwalde Block A', 'KW Jänschwalde Block B', 'KW Jänschwalde Block C',
                        'KW Jänschwalde Block D', 'KW Jänschwalde Block E', 'KW Jänschwalde Block F'],
    'Lippendorf 214': ['KW Lippendorf Block R', 'KW Lippendorf Block S'],
    'Neurath 238': ['Neurath A', 'Neurath B', 'Neurath C', 'Neurath D', 'Neurath E', 'Neurath F', 'Neurath G'],
    'Niederaußem 237': ['Niederaußem C', 'Niederaußem D', 'Niederaußem E', 'Niederaußem F',
                        'Niederaußem G', 'Niederaußem H', 'Niederaußem K (BoA 1)',
                        'Frimmersdorf P', 'Frimmersdorf Q',
                        'Weisweiler E', 'Weisweiler F', 'Weisweiler G', 'Weisweiler H'],
    'Schkopau 215': ['Schkopau A', 'Schkopau B'],
    'Buschhaus 213': ['Buschhaus'],
    'Schwarze Pumpe 313': ['KW Schwarze Pumpe Block A', 'KW Schwarze Pumpe Block B'],
    'Heilbronn 254': ['HKW Heilbronn Block 7'],
    'Rostock 304': ['Kraftwerk Rostock'], # incomplete,
    'Moorburg 319': ['HKW Moorburg Block A', 'HKW Moorburg Block B'],
    'Wilhelmshaven 204': ['Wilhelmshaven'],
    'Großkrotzenburg 272': ['Staudinger 5'], # incomplete
    'Bexbach 225': ['BEXBACH_A_GESAMT'],
    'Lippborg 248': ['BERGKAMEN_A'],
    'Ensdorf 948': ['Ensdorf 1', 'Ensdorf 3'],
    'Gersteinwerk 240': ['Gersteinwerk K2'],
    'Altbach 307': ['HKW Altbach/Deizisau Block 1', 'HKW Altbach/Deizisau Block 2'],
    'Reuter West 216': ['HKW Reuter West Block D', 'HKW Reuter West Block E'],
    'Kummerfeld 290': ['HKW Wedel Block 1', 'HKW Wedel Block 2'],
    'Ibbenbüren 309': ['Ibbenbüren B'],
    'Voerde 203': ['Kraftwerk Voerde Block A', 'Kraftwerk Voerde Block B'],
    'Scholven 243': ['Scholven B', 'Scholven C'], # incomplete
    'Ingolstadt 263': ['Ingolstadt 3', 'Ingolstadt 4'],
    'Vierraden 302': ['IKS Schwedt SE1 Block 1', 'IKS Schwedt SE2 Block 2']
}

In [32]:
DE_name_values = {name for list_of_names in DE_name_map.values() for name in list_of_names}

Check that the keys are part of the model's names:

In [33]:
assert set(DE_name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [34]:
assert DE_name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's generators not in the list:

In [35]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if gen['country'] == 'DE' and gen['name'] not in DE_name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
65,Altenkleusheim 245,hydro_pure_ps,140.0
27,Altentreptow/Süd 972,Gas,75.0
25,BASF 270,fossil_coal_gas,460.0
106,BASF 271,fossil_coal_hard,870.0
103,Bergrheinfeld 281,hydro_pure_ps,160.0
...,...,...,...
0,Wengerohr 228,Hydro,16.4
78,Wengerohr 229,Hydro,13.6
26,Windpark Iven 954,Gas,15.1
85,Wolmirstedt 292,Gas,52.0


ENTSO-E generators not in the list:

In [36]:
entsoe_data[(entsoe_data['Country'] == 'DE') & (~entsoe_data['PowerSystemResourceName'].isin(DE_name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
169,Block 1,Fossil Hard coal,136.0
170,Block 2,Fossil Hard coal,136.0
171,Block 3,Fossil Hard coal,690.0
172,Block AGuD,Fossil Gas,100.0
173,Block B,Fossil Gas,100.0
...,...,...,...
1659,Weisweiler VGT - BI. G,Fossil Gas,200.0
1660,Weisweiler VGT - BI. H,Fossil Gas,200.0
1662,Westfalen C,Fossil Hard coal,250.0
1663,Westfalen E,Fossil Hard coal,763.7


### Nuclear plants in other countries

In [37]:
jaslovske = [name for name in all_gen_names if name[-3:] == '377'][0]
name_map = CH_name_map | FR_name_map | ES_name_map | DE_name_map | {
    jaslovske: ['Bohunice TG31', 'Bohunice TG32', 'Bohunice TG41', 'Bohunice TG42'],
    'Borssele 846': ['Borssele 30'],
    'Tihange 521': ['TIHANGE 1N', 'TIHANGE 1S', 'TIHANGE 2', 'TIHANGE 3'],
    'Mochovce 378': ['Mochovce TG11', 'Mochovce TG12', 'Mochovce TG21', 'Mochovce TG22'],
    'Cernavoda 404': ['CNE_U1_RET_CA', 'CNE_U2_RET_CA'],
    'Kallo 515': ['DOEL 1', 'DOEL 2', 'DOEL 3', 'DOEL 4'],
    'Dukovany 176': ['EDUK_B1____', 'EDUK_B2____', 'EDUK_B3____', 'EDUK_B4____'],
    'Temelín 179': ['ETEM_G1____', 'ETEM_G2____'],
    'Kozloduy 418': ['NPP KZL G9', 'NPP KZL G10'],
    'Paks 782': ['PA_gép1', 'PA_gép2', 'PA_gép3', 'PA_gép4', 'PA_gép5', 'PA_gép6', 'PA_gép7', 'PA_gép8'],
}

In [38]:
name_values = {name for list_of_names in name_map.values() for name in list_of_names}

Total number of model generators, total number of ENTSO-E generators:

In [39]:
len(name_map), len(name_values)

(123, 275)

Check that the keys are part of the model's names:

In [40]:
assert set(name_map.keys()).issubset(all_gen_names)

Check that the names belong to the ENTSO-E names:

In [41]:
assert name_values.issubset(entsoe_data['PowerSystemResourceName'])

Model's nuclear generators not in the list:

In [42]:
pd.DataFrame([[gen['name'], gen['type'], 100 * gen['pmax']] for gen in model['gen'].values()
             if 'nuclear' in gen['type'] and gen['name'] not in name_map.keys()],
             columns=['Name', 'Type', 'Pmax']).sort_values('Name')

,Name,Type,Pmax
1,Grafenrheinfeld 282,nuclear_cons,1275.0
0,Krško 886,nuclear,696.0


ENTSO-E nuclear generators not in the list:

In [43]:
entsoe_data[(entsoe_data['ProductionType'] == 'Nuclear') & (~entsoe_data['PowerSystemResourceName'].isin(name_values))] \
    [['PowerSystemResourceName', 'ProductionType', 'InstalledGenCapacity']]

,PowerSystemResourceName,ProductionType,InstalledGenCapacity
1186,S.M.GAROÑA,Nuclear,455.2
1353,TRILLO,Nuclear,1003.4
1588,VANDELLOS,Nuclear,1045.3


### Update the model

Add 'entsoe_names' field to the generators in the model and delete the temporary name:

In [44]:
for gen in model['gen'].values():
    gen['entsoe_names'] = name_map[gen['name']] if gen['name'] in name_map.keys() else []
    del gen['name']

# Export improved model

Export:

In [45]:
model_json = json.dumps(model)
with open('../europe.json', 'w') as f:
    f.write(model_json)